# Importing useful libraries


In [1]:
import pandas as pd

# Loading the datasets

The datasets are big so we need to do some preprocessing step. We want to merge both datasets on the date and time attributes, and to keep only the 'Consommation' and 'Température' columns.

* the `weather` dataset can be downloaded on the internet via [Meteo-France](https://donneespubliques.meteofrance.fr/?fond=produit&id_produit=90&id_rubrique=32) 
* the `electricity` dataset can be downloaded on the internet via [RTE](https://odre.opendatasoft.com/explore/dataset/eco2mix-national-cons-def/table/?disjunctive.nature&sort=date_heure&dataChart=eyJxdWVyaWVzIjpbeyJjaGFydHMiOlt7InR5cGUiOiJsaW5lIiwiZnVuYyI6IlNVTSIsInlBeGlzIjoiY29uc29tbWF0aW9uIiwiY29sb3IiOiIjZWE1MjU0Iiwic2NpZW50aWZpY0Rpc3BsYXkiOnRydWV9XSwieEF4aXMiOiJkYXRlX2hldXJlIiwibWF4cG9pbnRzIjoyMDAsInRpbWVzY2FsZSI6Im1pbnV0ZSIsInNvcnQiOiIiLCJjb25maWciOnsiZGF0YXNldCI6ImVjbzJtaXgtbmF0aW9uYWwtY29ucy1kZWYiLCJvcHRpb25zIjp7ImRpc2p1bmN0aXZlLm5hdHVyZSI6dHJ1ZSwic29ydCI6Ii1kYXRlX2hldXJlIn19fV0sInRpbWVzY2FsZSI6IiIsImRpc3BsYXlMZWdlbmQiOnRydWUsImFsaWduTW9udGgiOnRydWV9)

In [2]:
weather = pd.read_csv("data/weather.csv", sep=';', low_memory=False)
electricity = pd.read_csv("data/electricity.csv", sep=';', low_memory=False)

In [3]:
weather.head(5)

,ID OMM station,Date,Pression au niveau mer,Variation de pression en 3 heures,Type de tendance barométrique,Direction du vent moyen 10 mn,Vitesse du vent moyen 10 mn,Température,Point de rosée,Humidité,...,Altitude,communes (name),communes (code),EPCI (name),EPCI (code),department (name),department (code),region (name),region (code),mois_de_l_annee
0,81405,2013-04-03T17:00:00+02:00,101460.0,NaN,NaN,40.0,7.2,302.75,293.85,59.0,...,4,Matoury,97307,CA du Centre Littoral,249730045.0,Guyane,973,Guyane,3.0,4
1,7027,2013-04-03T20:00:00+02:00,101180.0,-40.0,7.0,20.0,5.7,277.15,272.75,73.0,...,67,Carpiquet,14137,CU Caen la Mer,200065597.0,Calvados,14,Normandie,28.0,4
2,7181,2013-04-03T20:00:00+02:00,100890.0,20.0,3.0,70.0,7.7,278.25,271.15,60.0,...,336,Thuilley-aux-Groseilles,54523,CC du Pays de Colombey et du Sud Toulois,245400510.0,Meurthe-et-Moselle,54,Grand Est,44.0,4
3,7747,2013-04-03T20:00:00+02:00,100350.0,-100.0,6.0,170.0,1.5,286.75,281.95,73.0,...,42,Perpignan,66136,CU Perpignan Méditerranée Métropole,200027183.0,Pyrénées-Orientales,66,Occitanie,76.0,4
4,7207,2013-04-03T23:00:00+02:00,100950.0,100.0,1.0,30.0,10.3,276.95,271.75,69.0,...,34,Bangor,56009,CC de Belle Ile en Mer,245600465.0,Morbihan,56,Bretagne,53.0,4


In [4]:
electricity.head(5)

,Périmètre,Nature,Date,Heure,Date et Heure,Consommation (MW),Prévision J-1 (MW),Prévision J (MW),Fioul (MW),Charbon (MW),...,Gaz - TAC (MW),Gaz - Cogénération (MW),Gaz - CCG (MW),Gaz - Autres (MW),Hydraulique - Fil de l'eau + éclusée (MW),Hydraulique - Lacs (MW),Hydraulique - STEP turbinage (MW),Bioénergies - Déchets (MW),Bioénergies - Biomasse (MW),Bioénergies - Biogaz (MW)
0,France,Données définitives,2014-04-20,11:30,2014-04-20T11:30:00+02:00,48494.0,46600,49000,239.0,-23.0,...,3.0,346,155.0,51.0,4534.0,2004.0,999.0,422.0,165.0,181.0
1,France,Données définitives,2014-04-20,14:30,2014-04-20T14:30:00+02:00,44841.0,44500,45100,240.0,-25.0,...,4.0,346,214.0,51.0,3705.0,1583.0,184.0,419.0,163.0,179.0
2,France,Données définitives,2014-04-20,16:45,2014-04-20T16:45:00+02:00,NaN,40200,40950,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,France,Données définitives,2014-04-20,17:00,2014-04-20T17:00:00+02:00,40796.0,40000,40800,241.0,-23.0,...,3.0,346,220.0,51.0,3642.0,1422.0,0.0,424.0,168.0,184.0
4,France,Données définitives,2014-04-20,22:30,2014-04-20T22:30:00+02:00,47951.0,48300,48400,239.0,-24.0,...,3.0,346,196.0,51.0,5271.0,2384.0,1249.0,422.0,152.0,176.0


In [5]:
# Parse dates
def parse_date(date):
    """
    Parse a date string into two parts.
    ex: parse_date("2013-04-03T17:00:00+02:00") = "2013-04-03", "17:00"
    """
    date = date.split("T")
    return date[0], date[1].split("+")[0][:5]

weather["Date"], weather["Heure"] = zip(*weather["Date"].apply(parse_date))
# Keep only the temperatures and the date
weather2 = weather[["Date", "Heure", "Température"]]
# Keep only the consumption and the date
electricity2 = electricity[["Date", "Heure", "Consommation (MW)"]]

In [6]:
# Sort the data by date and hour
electricity3 = electricity2.sort_values(by=["Date", "Heure"])
weather3 = weather2.sort_values(by=["Date", "Heure"])

In [7]:
# Compute the average temperature for each date and hour
weather4 = weather3.groupby(["Date", "Heure"]).mean().reset_index()

In [8]:
# Merge the two dataframes and add NaN values when the temperature is missing
df = pd.merge(electricity3, weather4, on=["Date", "Heure"], how="outer")

# Remove the rows when the consumption is missing
df = df.dropna(subset=["Consommation (MW)"])

# Convert temperatures from Kelvin to Celsius
def convert_from_kelvin_to_celcius(temperature):
    return temperature - 273.15

df["Température"] = df["Température"].apply(convert_from_kelvin_to_celcius)

# Set the date and hour as index
df = df.set_index(pd.to_datetime(df["Date"] + " " + df["Heure"]))

# Interpolate the missing temperatures
df["Température"] = df["Température"].interpolate(method='time')

# Add column for the exponentially smoothed temperature of factor 0.95
df['Temp95'] = df['Température'].ewm(alpha=0.95).mean()

# Add column for the exponentially smoothed temperature of factor 0.99
df['Temp99'] = df['Température'].ewm(alpha=0.99).mean()

# Add column for the minimal temperature of the day
df['TempMin99'] = df.groupby(df.index.date)['Temp99'].transform('min')

# Add column for the maximal temperature of the day
df['TempMax99'] = df.groupby(df.index.date)['Temp99'].transform('max')

# Rename the column
df['Consommation'] = df['Consommation (MW)']
df = df.drop(columns=['Consommation (MW)'])

# Add column for the Consumption of the previous day
df['Consommation1'] = df['Consommation'].shift(1)

# Add column for the Consumption of the previous week
df['Consommation7'] = df['Consommation'].shift(7)

# Add column that gives the row number
df['Index'] = df.reset_index().index

# Add column for the Time of Year : value grows linearly from 0 on the 1st of January 00h00 to 1 on the 31 st of December 23h30
df['TimeOfYear'] = (df.index.dayofyear - 1) / 364 + (df.index.hour + df.index.minute / 60) / 8760

# DayType is a categorical variable indicating the type of the day of the week
df['DayType'] = df.index.dayofweek

# Add column for the Daylight Saving Time
df['DLS'] = (df.index.month >= 4) & (df.index.month <= 10)

# Remove the rows with missing values
df = df.dropna()

# Export the data
df.to_csv("data/interpolated.csv")

We want to train the models on the periods that were initially used in [this paper](https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=9382417) : 

* the train set ranges from the beginning of 2012 to the end of August 2019
* the test set is divided into two parts : the first part ranges from March 16th to April 15th 2020, and the second from April 16th to June 7th 2020.

In [9]:
df.head(10)

,Date,Heure,Température,Temp95,Temp99,TempMin99,TempMax99,Consommation,Consommation1,Consommation7,Index,TimeOfYear,DayType,DLS
2012-01-01 03:30:00,2012-01-01,03:30,13.382456,13.383102,13.382580,12.370438,15.672062,50664.0,52496.0,58315.0,7,0.000400,6,False
2012-01-01 04:00:00,2012-01-01,04:00,13.370175,13.370822,13.370299,12.370438,15.672062,49161.0,50664.0,58315.0,8,0.000457,6,False
2012-01-01 04:30:00,2012-01-01,04:30,13.432330,13.429255,13.431710,12.370438,15.672062,47596.0,49161.0,56231.0,9,0.000514,6,False
2012-01-01 05:00:00,2012-01-01,05:00,13.494485,13.491223,13.493857,12.370438,15.672062,46656.0,47596.0,56075.0,10,0.000571,6,False
2012-01-01 05:30:00,2012-01-01,05:30,13.556639,13.553369,13.556012,12.370438,15.672062,46277.0,46656.0,55532.0,11,0.000628,6,False
2012-01-01 06:00:00,2012-01-01,06:00,13.618794,13.615523,13.618166,12.370438,15.672062,45827.0,46277.0,54911.0,12,0.000685,6,False
2012-01-01 06:30:00,2012-01-01,06:30,13.680949,13.677677,13.680321,12.370438,15.672062,45897.0,45827.0,52496.0,13,0.000742,6,False
2012-01-01 07:00:00,2012-01-01,07:00,13.743103,13.739832,13.742476,12.370438,15.672062,45580.0,45897.0,50664.0,14,0.000799,6,False
2012-01-01 07:30:00,2012-01-01,07:30,13.730860,13.731309,13.730976,12.370438,15.672062,45637.0,45580.0,49161.0,15,0.000856,6,False
2012-01-01 08:00:00,2012-01-01,08:00,13.718617,13.719251,13.718740,12.370438,15.672062,45770.0,45637.0,47596.0,16,0.000913,6,False


In [10]:
# Train set ranges from the beginning of 2012 to the end of August 2019.
df_train = df[df["Date"] < "2019-09-01"]

# Divide the crisis test data in two periods. 
# The first one ranges from March 16 th to April 15th 2020
df_test1 = df[(df["Date"] >= "2020-03-16") & (df["Date"] < "2020-04-16")]

# and the second one from April 16th to June 7th 2020.
df_test2 = df[(df["Date"] >= "2020-04-16") & (df["Date"] < "2020-06-08")]

# Export the sets
df_train.to_csv("data/train.csv", index=False)
df_test1.to_csv("data/test1.csv", index=False)
df_test2.to_csv("data/test2.csv", index=False)